# Advent of code 2023

Solutions are my own, if any external source including hints have been used it shall be mentioned and linked.


## Part1

--- Day 10: Pipe Maze ---



The pipes are arranged in a two-dimensional grid of tiles:

    | is a vertical pipe connecting north and south.
    - is a horizontal pipe connecting east and west.
    L is a 90-degree bend connecting north and east.
    J is a 90-degree bend connecting north and west.
    7 is a 90-degree bend connecting south and west.
    F is a 90-degree bend connecting south and east.
    . is ground; there is no pipe in this tile.
    S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.



In [1]:
from __future__ import annotations
from dataclasses import dataclass
from collections import deque



PIPES = {
    '|': [(0,1), (0,-1)], # deltas up or down
    '-' : [(1,0), (-1,0)],
    'L' : [(1,1),(-1,-1)],
    'J' : [(1,-1), (-1, 1)],
    '7' : [(-1,-1), (1,1)],
    'F' : [(-1,1), (1, -1)],
    'S' : [0]
}

@dataclass
class Pipe:
    symbol:str
    xy: tuple[int]

    @staticmethod
    def parse_pipe(char:str, xy:tuple[int])->Pipe:
        return Pipe(symbol=char,
                    xy=xy)
    


@dataclass
class Network:
    network:dict[tuple, Pipe] # NOTE: x, y here might be overkill
    xmax:int
    ymax:int
    
    def _find_start(self):
        for xy, val in self.network.items():
            if val.symbol == 'S':
                return xy
    
    def __post_init__(self):
        self.start = self._find_start()

    def search(self):
        """
        bfs
        """
        seen = {self.start}
        queue = deque([self.start])
        
        while queue:
            x, y = queue.popleft()
            current_pipe = self.network.get((x,y))
            # needed help with logic
            # going up
            up = (x, y-1)
            symbol = "." if not self.network.get(up) else self.network.get(up).symbol
            # in bounds                   # curr can go up      # receive upward movement       # not in seen/ visited
            if y > 0 and  current_pipe.symbol in "S|JL" and symbol in "|7F" and up not in seen:
                # print("up")
                seen.add(up)
                queue.append(up)
            # going down
            down = (x, y+1)
            symbol = "." if not self.network.get(down) else self.network.get(down).symbol
            # in bounds               #  curr can go down                #  can receive downward   
            if y < self.ymax - 1 and current_pipe.symbol in "S|7F" and symbol in "|JL" and down not in seen:
                # print('down')
                seen.add(down)
                queue.append(down)
            # going left
            left = (x-1, y)
            symbol = "." if not self.network.get(left) else self.network.get(left).symbol
            # in bounds                  # curr able to go left         # can receive from left 
            if x > 0 and current_pipe.symbol in "S-J7" and symbol in "-LF" and left not in seen:
                # print('left')
                seen.add(left)
                queue.append(left)
            # going right
            right = (x+1, y)
            symbol = "." if not self.network.get(right) else self.network.get(right).symbol
            # in bounds
            if x < self.xmax - 1 and current_pipe.symbol in "S-LF" and symbol in "-J7" and right not in seen:
                # print('right')
                seen.add(right)
                queue.append(right)
        self.seen = seen
      

    @staticmethod
    def parse_network(puzzle:str)->Network:
        rows = puzzle.splitlines()
        xmax, ymax = len(rows[0]), len(rows)
        network = dict()
        for y, row in enumerate(rows):
            for x, char in enumerate(row):
                if char in PIPES:
                    pipe = Pipe(
                        symbol = char,
                        xy = (x, y)
                    )
                    network[(x,y)] = pipe
        return Network(network=network, 
                       xmax=xmax, 
                       ymax=ymax)


TEST = """-L|F7
7S-7|
L|7||
-L-J|
L|-JF"""

TEST2="""7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ"""


def part1(puzzle:str)->int:
    network = Network.parse_network(puzzle=puzzle)
    network.search()
    return len(network.seen) // 2


assert part1(puzzle=TEST) == 4
assert part1(puzzle=TEST2) == 8


## Solutions

In [3]:
with open("puzzle_input/day10.txt") as file:
    puzzle = file.read()
print(part1(puzzle=puzzle))

6923
